In [1]:
import os
import re
import csv
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
audio_features = pd.read_csv('audio_features.csv')
audio_features.drop_duplicates(inplace=True)

In [3]:
audio_features_files = audio_features['file_name']

In [4]:
def list_files_in_directory(directory_path):
    try:
        files = [f for f in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, f))]
        return files
    except Exception as e:
        return f"An error occurred: {e}"


In [5]:
parsed_files = list_files_in_directory('C:/Users/victor.serra/OneDrive - americanas s.a/Área de Trabalho/python/automacao/imagens_audio/melspectrogram')
parsed_files_base = [os.path.splitext(f)[0] for f in parsed_files]

In [6]:
chords = pd.read_excel('C:/Users/victor.serra/OneDrive - americanas s.a/Área de Trabalho/python/chords.xlsx')

In [7]:
audio_list = pd.read_csv('C:/Users/victor.serra/OneDrive - americanas s.a/Área de Trabalho/python/audio_list.csv')

In [8]:
chords

,Tone,Chord,Chord Name,Chord Family,Alternate Symbols
0,C,C,C major,NaN,"['C', 'CM']"
1,C,Cm,C minor,m,"['Cm', 'Cmin', 'C-']"
2,C,C6,C sixth,6,"['C6', 'Cmajor6', 'CMaj6', 'CM6']"
3,C,Cm6,C minor sixth,m6,"['Cm6', 'Cminor6', 'Cmin6', 'C-6']"
4,C,C6/9,C sixth ninth,6/9,"['C6/9', 'C6(add9)', 'CMaj6(add9)', 'CM6(add9)']"
...,...,...,...,...,...
548,Bb,Bb9sus4,B flat ninth suspended fourth,9sus4,['Bb9sus4']
549,Bb,Bb9sus2,B flat ninth suspended second,9sus2,['Bb9sus2']
550,Bb,Bbaug,B flat augmented,aug,['Bbaug']
551,Bb,Bbdim,B flat diminished,dim,['Bbdim']


In [9]:
audio_list[30000:].head()

# se conter em audio_id a informacao musicguitar ou mp

,chord,audio_id,clean_audio_id,audio_url,tunning_description,guitar_string,id_chord
30000,F9b5,musicguitarF7b5_Gfastx03423DGCFADmp3,guitarF7b5_Gx03423DGCFAD,/chord-sound/guitar-F7b5_G-fast-x-0-3-4-2-3-DG...,Full Step Down,DGCFAD,66
30001,F9b5,musicpianopianoF9b5fastfabebgmp3,pianopianoF9b5fabebg,/chord-sound/piano-F9b5-fast-f-a-b-eb-g.mp3,Full Step Down,DGCFAD,66
30002,F9b5,musicukuleleF9b5fast10111110mp3,ukuleleF9b510111110,/chord-sound/ukulele-F9b5-fast-10-11-11-10.mp3,Full Step Down,DGCFAD,66
30003,F9b5,musicviolinF9b5fastx323mp3,violinF9b5x323,/chord-sound/violin-F9b5-fast-x-3-2-3.mp3,Full Step Down,DGCFAD,66
30004,F9b5,musiccelloF9b5fastx10910mp3,celloF9b5x10910,/chord-sound/cello-F9b5-fast-x-10-9-10.mp3,Full Step Down,DGCFAD,66


In [10]:
filtered_audio_list = audio_list[audio_list['audio_id'].str.contains('guitar')]
filtered_audio_list

,chord,audio_id,clean_audio_id,audio_url,tunning_description,guitar_string,id_chord
0,C,musicguitarCslowx32010mp3,guitarCx32010,/chord-sound/guitar-C-slow-x-3-2-0-1-0.mp3,standard,EADGBE,0
1,C,musicguitarCfastx32010mp3,guitarCx32010,/chord-sound/guitar-C-fast-x-3-2-0-1-0.mp3,standard,EADGBE,0
2,C,musicguitarCstrumx32010mp3,guitarCx32010,/chord-sound/guitar-C-strum-x-3-2-0-1-0.mp3,standard,EADGBE,0
3,C,musicguitarCslow81010988mp3v,guitarC81010988v,/chord-sound/guitar-C-slow-8-10-10-9-8-8.mp3,standard,EADGBE,0
4,C,musicguitarCfast81010988mp3v,guitarC81010988v,/chord-sound/guitar-C-fast-8-10-10-9-8-8.mp3,standard,EADGBE,0
...,...,...,...,...,...,...,...
61364,B5,musicguitarB5strum91111x99DGCFADmp3,guitarB591111x99DGCFAD,/chord-sound/guitar-B5-strum-9-11-11-x-9-9-DGC...,Full Step Down,DGCFAD,268
61365,B5,musicguitarB5slow91111x99DGCFADmp3v,guitarB591111x99DGCFADv,/chord-sound/guitar-B5-slow-9-11-11-x-9-9-DGCF...,Full Step Down,DGCFAD,268
61366,B5,musicguitarB5fast91111x99DGCFADmp3v,guitarB591111x99DGCFADv,/chord-sound/guitar-B5-fast-9-11-11-x-9-9-DGCF...,Full Step Down,DGCFAD,268
61367,B5,musicguitarB5strum91111x99DGCFADmp3v,guitarB591111x99DGCFADv,/chord-sound/guitar-B5-strum-9-11-11-x-9-9-DGC...,Full Step Down,DGCFAD,268


In [12]:
filtered_audio_list = filtered_audio_list[['chord','guitar_string','clean','audio_id']]
filtered_audio_list['clean'] = filtered_audio_list.apply(lambda row: row['clean'].replace(row['guitar_string'], '', 1), axis=1)

C:\Users\victor.serra\AppData\Local\Temp\ipykernel_11260\3809266093.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_audio_list['clean'] = filtered_audio_list.apply(lambda row: row['clean'].replace(row['guitar_string'], '', 1), axis=1)


In [13]:
files_names = pd.DataFrame(audio_features_files)
files_names = files_names[files_names['file_name'].str.contains('guitar')]

In [14]:
files_names['tuning'] = files_names['file_name'].str.extract(r't_([A-Za-z0-9]+)_')
files_names['details'] = files_names['file_name'].str.extract(r't_[A-Za-z0-9]+_(.+)\.mp3')
files_names['details'] = files_names.apply(lambda row: row['details'].replace(row['tuning'], '', 1), axis=1)
files_names['details'] = files_names['details'].apply(lambda x: re.sub(r'music|guitar', '',x))

In [51]:
result = pd.merge(files_names, filtered_audio_list[['clean', 'chord']], left_on='details', right_on='clean', how='left')
result.drop_duplicates(inplace=True)

duplicated_files = result[result['file_name'].duplicated(keep=False)]

filtered_duplicated = duplicated_files[duplicated_files.apply(lambda row: row['chord'] in row['file_name'], axis=1)]

result = result[~result['file_name'].duplicated(keep=False)]
result = pd.concat([result, filtered_duplicated])

In [52]:
result.to_csv('labels.csv', index=False)
result

,file_name,tuning,details,clean,chord
0,t_DADGBE_guitarA(b5)sus_Cfastxx108108DADGBE.mp3,DADGBE,A(b5)sus_Cfastxx108108,A(b5)sus_Cfastxx108108,Adim
2,t_DADGBE_guitarA(b5)sus_Fsfast404242DADGBE.mp3,DADGBE,A(b5)sus_Fsfast404242,A(b5)sus_Fsfast404242,D#dim
5,t_DADGBE_guitarA11sus2_Cfast1077787DADGBE.mp3,DADGBE,A11sus2_Cfast1077787,A11sus2_Cfast1077787,Am11
6,t_DADGBE_guitarA11sus2_Fsfast400000DADGBE.mp3,DADGBE,A11sus2_Fsfast400000,A11sus2_Fsfast400000,Em11
10,t_DADGBE_guitarA13fast004020DADGBE.mp3,DADGBE,A13fast004020,A13fast004020,A13
...,...,...,...,...,...
55310,t_EADGBE_guitarGsus2strum3x0233v.mp3,EADGBE,Gsus2strum3x0233v,Gsus2strum3x0233v,Gsus2
55312,t_EADGBE_guitarGsus2strumx1012121010v.mp3,EADGBE,Gsus2strumx1012121010v,Gsus2strumx1012121010v,Gsus2
55316,t_EADGBE_guitarGsus2strumx1077810v.mp3,EADGBE,Gsus2strumx1077810v,Gsus2strumx1077810v,Gsus2
55320,t_EADGBE_guitarGsus2strumxx5233v.mp3,EADGBE,Gsus2strumxx5233v,Gsus2strumxx5233v,Gsus2


In [54]:
# conte as linhas distintas agrupado por chord
result.groupby('chord').count()

,file_name,tuning,details,clean
chord,,,,
A,83,83,83,83
A#,90,90,90,90
A#13,109,109,109,109
A#5,41,41,41,41
A#6,100,100,100,100
...,...,...,...,...
Gmaj13,127,127,127,127
Gmaj7,138,138,138,138
Gmaj9,108,108,108,108
